# RNN

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, GRU, Input
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

## Data

In [ ]:
# Cargar los datos
data = pd.read_csv('./datasets/unified.csv')  # Cambia 'dataset.csv' por tu archivo

# Separar inputs y etiquetas
X = data[['input1', 'input2', 'input3']].values  # Inputs
y = data[['label']].values  # Asegúrate de que ya sea un vector one-hot

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar las dimensiones para la RNN
X_train = X_train.reshape((X_train.shape[0], 3, 1))
X_test = X_test.reshape((X_test.shape[0], 3, 1))


## Model

### Simple RNN

In [ ]:
model = Sequential([
    Input(shape=(3, 1007)),
    SimpleRNN(32, activation='tanh', input_shape=(3, 1007)),  # 3 timesteps, 1 feature
    Dense(1007, activation='softmax')  # Cambia softmax según el tipo de tarea
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/home/carles/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### LSTM

In [ ]:
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(3, 1007), return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1007, activation='softmax')  # 1007 clases en la salida
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'Input' is not defined

### GRU

In [ ]:
model = Sequential([
    Input(shape=(3, 1007)),
    GRU(64, activation='tanh', input_shape=(3, 1007), return_sequences=False),  # GRU en lugar de SimpleRNN
    Dense(32, activation='relu'),
    Dense(1007, activation='softmax')  # Cambia softmax según el tipo de tarea
])
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/home/carles/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Training

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


## Testing

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Visualizar el rendimiento durante el entrenamiento
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()